# Explanation of the main script

This tutorial uses freely available HapMap data: `hapmap3_r3_b36_fwd.consensus.qc`. We simulated a binary outcome measure (i.e., a binary phenotypic trait) and added this to the dataset. The outcome measure was only simulated for the founders in the HapMap data. This data set will be referred to as `HapMap_3_r3_1`. 

The HapMap data, without our simulated outcome measure, can also be obtained from http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/2010-05_phaseIII/plink_format/.

It is essential for the execution of the tutorial that that all scripts belonging to this tutorial are in the same directory on your UNIX workstation.

Many scripts include comments which explain how these scripts work. Note, in order to complete the tutorial it is essential to execute all commands in this tutorial.

This script can also be used for your own data analysis, to use it as such, replace the name of the HapMap file with the name of your own data file. 

Furthermore, this script is based on a binary outcome measure, and is therefore not applicable for quantitative outcome measures (this would require some adaptations)

Note, most GWAS studies are performed on an ethnic homogenous population, in which population outliers are removed. The HapMap data, used for this tutorial, contains multiple distinct ethnic groups, which makes it problematic for analysis.

Therefore, we have selected only the EUR individuals of the complete HapMap sample for the tutorials 1-3. This selection is already performed in the `HapMap_3_r3_1` file from our GitHub page.

The Rscripts used in this tutorial are all executed from the Unix command line.

Therefore, this tutorial and the other tutorials from our GitHub page, can be completed simply by copy-and-pasting all commands from the 'main scripts' into the Unix terminal.

For a thorough theoretical explanation of all QC steps we refer to the article accompanying this tutorial entitled "A tutorial on conducting Genome-Wide-Association Studies: Quality control and statistical analysis (https://www.ncbi.nlm.nih.gov/pubmed/29484742).

## Setup

> **Author's note:** below is some setup code I need to make the notebook shine a bit more. Some imports and styling for the plots later on.

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

plt.style.use('ggplot')

## Step 1

Investigate missingness per individual and per SNP and make histograms.

> **Author's note:** Info about the columns of:
>    - the `.bed` file can be found here https://www.cog-genomics.org/plink/1.9/formats#bed
>    - and `.bim` file can be found here https://www.cog-genomics.org/plink/1.9/formats#bim
>    - and `.fam` file can be found here https://www.cog-genomics.org/plink/1.9/formats#fam

In [ ]:
!plink --bfile HapMap_3_r3_1 --missing

Output: `plink.imiss` and `plink.lmiss`, these files show respectively the proportion of missing SNPs per individual and the proportion of missing individuals per SNP.

> **Author's note:** Info about the columns of:
>    - the `.imiss` file can be found here https://www.cog-genomics.org/plink/1.9/formats#imiss
>    - and `.lmiss` file can be found here https://www.cog-genomics.org/plink/1.9/formats#lmiss

Generate plots to visualize the missingness results.

In [ ]:
# Read data into DataFrame.
indmiss = pd.read_csv('plink.imiss', sep='\s+')
snpmiss = pd.read_csv('plink.lmiss', sep='\s+')

# Plotting.
fig, axs = plt.subplots(1, 2, figsize=(15, 5))  #  Side-by-side figure.

indmiss[['F_MISS']].plot(
    kind='hist',
    title='Histogram individual missingness',
    ax=axs[0])

snpmiss[['F_MISS']].plot(
    kind='hist',
    title='Histogram sample missingness',
    ax=axs[1])

Delete SNPs and individuals with high levels of missingness, explanation of this and all following steps can be found in box 1 and table 1 of the article mentioned in the comments of this script.

The following two QC commands will not remove any SNPs or individuals. However, it is good practice to start the QC with these non-stringent thresholds.  

Delete SNPs with missingness $>0.2$.

In [ ]:
!plink --bfile HapMap_3_r3_1 --geno 0.2 --make-bed --out HapMap_3_r3_2

Delete individuals with missingness $>0.2$.

In [ ]:
!plink --bfile HapMap_3_r3_2 --mind 0.2 --make-bed --out HapMap_3_r3_3

Delete SNPs with missingness $>0.02$.

In [ ]:
!plink --bfile HapMap_3_r3_3 --geno 0.02 --make-bed --out HapMap_3_r3_4

Delete individuals with missingness $>0.02$.

In [ ]:
!plink --bfile HapMap_3_r3_4 --mind 0.02 --make-bed --out HapMap_3_r3_5

## Step 2

Check for sex discrepancy.

Subjects who were a priori determined as females must have a F value of $<0.2$, and subjects who were a priori determined as males must have a F value $>0.8$. This F value is based on the X chromosome inbreeding (homozygosity) estimate.

Subjects who do not fulfil these requirements are flagged "PROBLEM" by PLINK.

In [ ]:
!plink --bfile HapMap_3_r3_5 --check-sex

> **Author's note:** Info about the columns of the `.sexcheck` file can be found here https://www.cog-genomics.org/plink/1.9/formats#sexcheck

Generate plots to visualize the sex-check results.

In [ ]:
# Read data into DataFrame.
gender = pd.read_csv('plink.sexcheck', sep='\s+')

# Plotting.
fig, axs = plt.subplots(2, 2, figsize=(15, 10))  #  4x4 figure.

gender['F'].plot(
    kind='hist',
    title='Gender',
    ax=axs[0, 0])

gender.loc[lambda d: d['PEDSEX']==1, 'F'].plot(
    kind='hist',
    title='Men',
    ax=axs[0, 1])

gender.loc[lambda d: d['PEDSEX']==2, 'F'].plot(
    kind='hist',
    title='Female',
    ax=axs[1, 0])

These checks indicate that there is one woman with a sex discrepancy, F value of 0.99. (When using other datasets often a few discrepancies will be found).

The following two scripts can be used to deal with individuals with a sex discrepancy.

Note, please use one of the two options below to generate the bfile `hapmap_r23a_6`, this file we will use in the next step of this tutorial.

1\) Delete individuals with sex discrepancy.

This command generates a list of individuals with the status "PROBLEM".

In [ ]:
!grep "PROBLEM" plink.sexcheck| awk '{print$1,$2}'> sex_discrepancy.txt

This command removes the list of individuals with the status "PROBLEM".

In [ ]:
!plink --bfile HapMap_3_r3_5 --remove sex_discrepancy.txt --make-bed --out HapMap_3_r3_6 

2\) impute-sex.

This imputes the sex based on the genotype information into your data set.

> **Author's note:** I think the following was meant to be run exclusively from the cell above. You either impute the problematic sample's sex or you remove the sample entirely.

In [ ]:
# !plink --bfile HapMap_3_r3_5 --impute-sex --make-bed --out HapMap_3_r3_6

## Step 3

Generate a bfile with autosomal SNPs only and delete SNPs with a low minor allele frequency (MAF).

Select autosomal SNPs only (i.e., from chromosomes 1 to 22).

In [ ]:
!awk '{ if ($1 >= 1 && $1 <= 22) print $2 }' HapMap_3_r3_6.bim > snp_1_22.txt
!plink --bfile HapMap_3_r3_6 --extract snp_1_22.txt --make-bed --out HapMap_3_r3_7

Generate a plot of the MAF distribution.

> **Author's note:** Info about the columns of the `.frq` can be found here https://www.cog-genomics.org/plink/1.9/formats#frq

In [ ]:
!plink --bfile HapMap_3_r3_7 --freq --out MAF_check

In [ ]:
# Read data into DataFrame.
maf_freq = pd.read_csv('MAF_check.frq', sep='\s+')

# Plotting.
maf_freq['MAF'].plot(
    kind='hist',
    title='MAF distribution')

Remove SNPs with a low MAF frequency.

In [ ]:
!plink --bfile HapMap_3_r3_7 --maf 0.05 --make-bed --out HapMap_3_r3_8

1073226 SNPs are left.

A conventional MAF threshold for a regular GWAS is between 0.01 or 0.05, depending on sample size.

## Step 4

Delete SNPs which are not in Hardy-Weinberg equilibrium (HWE).

Check the distribution of HWE p-values of all SNPs.

> **Author's note:** Info about the columns of the `.hwe` can be found here https://www.cog-genomics.org/plink/1.9/formats#hwe

In [ ]:
!plink --bfile HapMap_3_r3_8 --hardy

Selecting SNPs with HWE *p*-value below 0.00001, required for one of the two plot generated by the next *cell*, allows to zoom in on strongly deviating SNPs. 

In [ ]:
!awk '{ if ($9 <0.00001) print $0 }' plink.hwe>plinkzoomhwe.hwe

In [ ]:
# Read data into DataFrame.
hwe = pd.read_csv('plink.hwe', sep='\s+')
hwe_zoom = pd.read_csv('plinkzoomhwe.hwe', header=None, sep='\s+')
hwe_zoom.columns = hwe.columns

fig, axs = plt.subplots(1, 2, figsize=(15, 5))
# Plotting.
hwe['P'].plot(
    kind='hist',
    title='Histrogram HWE',
    ax=axs[0])

hwe_zoom['P'].plot(
    kind='hist',
    title='Histogram HWE: strongly deviating SNPs only',
    ax=axs[1])

By default the `--hwe` option in plink only filters for controls.

Therefore, we use two steps, first we use a stringent HWE threshold for controls, followed by a less stringent threshold for the case data.

In [ ]:
!plink --bfile HapMap_3_r3_8 --hwe 1e-6 --make-bed --out HapMap_hwe_filter_step1

The HWE threshold for the cases filters out only SNPs which deviate extremely from HWE. 

This second HWE step only focusses on cases because in the controls all SNPs with a HWE *p*-value < hwe `1e-6` were already removed

In [ ]:
!plink --bfile HapMap_hwe_filter_step1 --hwe 1e-10 --hwe-all --make-bed --out HapMap_3_r3_9

Theoretical background for this step is given in our accompanying article: https://www.ncbi.nlm.nih.gov/pubmed/29484742 .

## Step 5

Generate a plot of the distribution of the heterozygosity rate of your subjects.

And remove individuals with a heterozygosity rate deviating more than 3 sd from the mean.

Checks for heterozygosity are performed on a set of SNPs which are not highly correlated.

Therefore, to generate a list of non-(highly)correlated SNPs, we exclude high inversion regions (inversion.txt [High LD regions]) and prune the SNPs using the command `--indep-pairwise`.

The parameters `50 5 0.2` stand respectively for: the window size, the number of SNPs to shift the window at each step, and the multiple correlation coefficient for a SNP being regressed on all other SNPs simultaneously.

> **Author's note:** Info about the columns of the `.prune.in` and `.prune.out` file can be found here https://www.cog-genomics.org/plink/1.9/ld

> <span style="color: #e67e22">**Critical implementation note:**</span> the original script has the argument `--range` in the cell below. Newer versions of PLINK v1.9 have deprecated this and therefore doesn't work as expected. through my readings, it seems that this argument defaults to a target of $[0, \infty]$ so I think it's safe to remove it. I did try `--extract range` but it required a file of which the contents I am unsure.

In [ ]:
!plink --bfile HapMap_3_r3_9 --exclude inversion.txt --indep-pairwise 50 5 0.2 --out indepSNP

Note, don't delete the file `indepSNP.prune.in`, we will use this file in later steps of the tutorial.

In [ ]:
!plink --bfile HapMap_3_r3_9 --extract indepSNP.prune.in --het --out R_check

This file contains your pruned data set.

Plot of the heterozygosity rate distribution

> **Author's note:** Info about the columns of the `.het` file can be found here https://www.cog-genomics.org/plink/1.9/formats#het

In [ ]:
# Read data into DataFrame.
het = pd.read_csv('R_check.het', sep='\s+')

# Plotting.
het['HET_RATE'] = (het['N(NM)'] - het['O(HOM)']) / het['N(NM)']
het['HET_RATE'].plot(
    kind='hist',
    title='Heterozygosity rate')

The following code generates a list of individuals who deviate more than 3 standard deviations from the heterozygosity rate mean.

For data manipulation we recommend using UNIX. However, when performing statistical calculations *Python* might be more convenient, hence the use of the *Python* for this step:

In [ ]:
mean = het['HET_RATE'].mean()
sdev = het['HET_RATE'].std()
het_fail = het.loc[lambda d: (
    (d['HET_RATE'] < (mean - 3*sdev)) |
    (d['HET_RATE'] > (mean + 3*sdev))
)]
het_fail['HET_DST'] = (het_fail['HET_RATE'] - het_fail['HET_RATE'].mean()) / (het_fail['HET_RATE'].std())
het_fail.to_csv('fail-het-qc.txt', sep='\t', index=False)

Output of the command above: `fail-het-qc.txt`.

When using our example data/the HapMap data this list contains 2 individuals (i.e., two individuals have a heterozygosity rate deviating more than 3 SD's from the mean).

Adapt this file to make it compatible for PLINK, by removing the header and selecting only the first two columns.

> <span style="color: #3498db">**Implementation note:**</span> I've totally changed how things are done in this subsection of the pipeline. It achieves the same thing ultimately but just makes more sense to me personally.

In [ ]:
!tail +2 fail-het-qc.txt | cut -f 1,2 > het_fail_ind.txt

Remove heterozygosity rate outliers.

In [ ]:
!plink --bfile HapMap_3_r3_9 --remove het_fail_ind.txt --make-bed --out HapMap_3_r3_10

## Step 6

It is essential to check datasets you analyse for cryptic relatedness.

Assuming a random population sample we are going to exclude all individuals above the pihat threshold of 0.2 in this tutorial.

Check for relationships between individuals with a pihat > 0.2.

 **Author's note:** Info about the columns of the `.genome` file can be found here https://www.cog-genomics.org/plink/1.9/formats#genome

In [ ]:
!plink --bfile HapMap_3_r3_10 --extract indepSNP.prune.in --genome --min 0.2 --out pihat_min0.2

The HapMap dataset is known to contain parent-offspring relations.

The following commands will visualize specifically these parent-offspring relations, using the z values. 

In [ ]:
!awk '{ if ($8 >0.9) print $0 }' pihat_min0.2.genome>zoom_pihat.genome

Generate a plot to assess the type of relationship.

In [ ]:
# Read data into DataFrame.
relatedness = pd.read_csv('pihat_min0.2.genome', sep='\s+')
relatedness_zoom = pd.read_csv('zoom_pihat.genome', sep='\s+')

# Plotting.
fig, axs = plt.subplots(2, 2, figsize=(15, 10))  #  4x4 figure.

# Top-left plot.
axs[0, 0].scatter(
    relatedness.loc[lambda d: d['RT'] == 'PO', 'Z0'],
    relatedness.loc[lambda d: d['RT'] == 'PO', 'Z1'],
    label='PO')
axs[0, 0].scatter(
    relatedness.loc[lambda d: d['RT'] == 'UN', 'Z0'],
    relatedness.loc[lambda d: d['RT'] == 'UN', 'Z1'],
    label='UN')
axs[0, 0].set_xlim(0, 1)
axs[0, 0].set_ylim(0, 1)
axs[0, 0].legend()

# Top-right plot.
axs[0, 1].scatter(
    relatedness_zoom.loc[lambda d: d['RT'] == 'PO', 'Z0'],
    relatedness_zoom.loc[lambda d: d['RT'] == 'PO', 'Z1'],
    label='PO')
axs[0, 1].set_xlim(0, 0.015)
axs[0, 1].set_ylim(0.975, 1)

# Bottom-left plot.
relatedness['PI_HAT'].plot(
    kind='hist',
    ax=axs[1, 0])

The generated plots show a considerable amount of related individuals (explentation plot; PO = parent-offspring, UN = unrelated individuals) in the Hapmap data, this is expected since the dataset was constructed as such.

Normally, family based data should be analyzed using specific family based methods. In this tutorial, for demonstrative purposes, we treat the relatedness as cryptic relatedness in a random population sample.

In this tutorial, we aim to remove all 'relatedness' from our dataset.

To demonstrate that the majority of the relatedness was due to parent-offspring we only include founders (individuals without parents in the dataset).

In [ ]:
!plink --bfile HapMap_3_r3_10 --filter-founders --make-bed --out HapMap_3_r3_11

Now we will look again for individuals with a pihat >0.2.

In [ ]:
!plink --bfile HapMap_3_r3_11 --extract indepSNP.prune.in --genome --min 0.2 --out pihat_min0.2_in_founders

The file `pihat_min0.2_in_founders.genome` shows that, after exclusion of all non-founders, only 1 individual pair with a pihat greater than 0.2 remains in the HapMap data.
This is likely to be a full sib or DZ twin pair based on the Z values. Noteworthy, they were not given the same family identity (FID) in the HapMap data.

For each pair of 'related' individuals with a pihat > 0.2, we recommend to remove the individual with the lowest call rate. 

In [ ]:
!plink --bfile HapMap_3_r3_11 --missing

Use `less` or `head` to check which individual has the highest call rate in the 'related pair'.

Generate a list of FID and IID of the individual(s) with a Pihat above 0.2, to check who had the lower call rate of the pair.

In our dataset the individual 13291  NA07045 had the lower call rate.

In [ ]:
!echo '13291\tNA07045' > 0.2_low_call_rate_pihat.txt

Delete the individuals with the lowest call rate in 'related' pairs with a pihat > 0.2 

In [ ]:
!plink --bfile HapMap_3_r3_11 --remove 0.2_low_call_rate_pihat.txt --make-bed --out HapMap_3_r3_12

---

## Congratulations

You've just succesfully completed the first tutorial! You are now able to conduct a proper genetic QC. 

For the next tutorial, using the *notebook*: `2_Main_script_MDS.ipynb`, you need the following files:

- The bfile `HapMap_3_r3_12` (i.e., `HapMap_3_r3_12.fam`, `HapMap_3_r3_12.bed`, and `HapMap_3_r3_12.bim`)
- indepSNP.prune.in